# Imports

In [1]:
import pandas as pd
from pathlib import Path

# File Finder 

In [2]:
BASE_DIR = Path("..")

# Check No of Files

In [3]:
silver_path = BASE_DIR / "data" / "silver"
parquet_files = list(silver_path.rglob("*.parquet"))

print(f"✅ {len(parquet_files)} Parquet Found\n")

✅ 21 Parquet Found



# Health check

In [4]:
results = []

for path in sorted(parquet_files):
    df = pd.read_parquet(path)
    nulls = df.isnull().sum().sum()
    
    results.append({
        "Layer":    path.parent.name,
        "Tabelle":  path.stem,
        "Rows":     df.shape[0],
        "Cols":     df.shape[1],
        "Nulls":    nulls,
        "Size KB":  round(path.stat().st_size / 1024, 1)
    })

overview = pd.DataFrame(results)
display(overview)

,Layer,Tabelle,Rows,Cols,Nulls,Size KB
0,dimensions,cities,37940,8,48988,1679.1
1,dimensions,colors,36,5,36,3.9
2,dimensions,countries,190,14,190,1106.4
3,dimensions,delivery_methods,10,5,10,3.8
4,dimensions,package_types,14,5,14,3.7
5,dimensions,payment_methods,4,5,4,3.6
6,dimensions,people,906,16,906,61.5
7,dimensions,provinces,53,10,54,332.3
8,dimensions,stock_groups,10,5,10,3.7
9,dimensions,stock_item_holdings,227,9,0,10.8


In [9]:
for path in sorted(parquet_files):
    df = pd.read_parquet(path)
    nulls = df.isnull().sum()
    nulls_only = nulls[nulls > 0]
    
    print(f"\n{'═'*60}")
    print(f"📄 {path.parent.name} / {path.stem}")
    print(f"{'═'*60}")
    print(f"Shape:  {df.shape[0]} rows x {df.shape[1]} cols")
    print(f"Dtypes:\n{df.dtypes}")
    
    if nulls_only.empty:
        print("\n✅ Keine Nulls")
    else:
        print(f"\n⚠️  Nulls:\n{nulls_only}")
    
    print(f"\nSample:\n{df.head(2)}")


════════════════════════════════════════════════════════════
📄 dimensions / cities
════════════════════════════════════════════════════════════
Shape:  37940 rows x 8 cols
Dtypes:
city_id                                int64
city_name                                str
state_province_id                      int64
location                                 str
latest_recorded_population             Int64
last_edited_by                         int64
valid_from                    datetime64[ns]
valid_to                      datetime64[ns]
dtype: object

⚠️  Nulls:
latest_recorded_population    11048
valid_to                      37940
dtype: int64

Sample:
   city_id   city_name  state_province_id  \
0        1  Aaronsburg                 39   
1        3      Abanda                  1   

                                         location  latest_recorded_population  \
0  0xE6100000010C07E11B542C73444087C09140035D53C0                         613   
1  0xE6100000010C4033880FEC8C4040EFBF3A33

In [10]:
# Nur Spalten die "date", "when", "time" im Namen haben aber noch str sind
suspicious = []

for path in sorted(parquet_files):
    df = pd.read_parquet(path)
    for col in df.select_dtypes(include="str").columns:
        if any(keyword in col.lower() for keyword in ["date", "when", "time", "from", "to"]):
            suspicious.append({
                "Tabelle": f"{path.parent.name}/{path.stem}",
                "Spalte":  col,
                "Dtype":   str(df[col].dtype)
            })

if suspicious:
    print("⚠️  Verdächtige str-Spalten:")
    for s in suspicious:
        print(f"  {s['Tabelle']} -> {s['Spalte']}")
else:
    print("✅ Alle Datumsspalten korrekt gecastet")


⚠️  Verdächtige str-Spalten:
  dimensions/provinces -> sales_territory
  dimensions/stock_groups -> stock_group_name
  dimensions/stock_items -> stock_item_name
  dimensions/stock_items -> custom_fields
  sales/customers -> customer_name
